In [1]:
import os
import glob
import re
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np

/home/nuno/GAIJ/sandbox/calaim/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
md_paths = '/home/nuno/GAIJ/sandbox/converted'
json_paths = '/home/nuno/GAIJ/sandbox/extracted'

In [3]:
# Use glob to list all .md files in the folder
markdown_files = glob.glob(os.path.join(md_paths, "*.md"))
np.random.shuffle(markdown_files)
os.makedirs(json_paths, exist_ok=True)

# model setup

In [4]:
# Specify the path to your LLaMA 3 model directory
import torch
model_path = '/home/nuno/GAIJ/sandbox/models/llama_32_1b_Instruct'

torch.cuda.empty_cache()

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Ensure the pad token is set
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

model = AutoModelForCausalLM.from_pretrained(model_path)
model.cpu()
torch.cuda.empty_cache()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.resize_token_embeddings(len(tokenizer))

# Set the pad_token_id and eos_token_id in model configuration
model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id

# Print token IDs to confirm they are integers
print("pad_token_id:", model.config.pad_token_id)
print("eos_token_id:", model.config.eos_token_id)

# Make sure eos_token_id and pad_token_id are valid integers
assert isinstance(model.config.pad_token_id, int), "pad_token_id should be an integer."
assert isinstance(model.config.eos_token_id, int), "eos_token_id should be an integer."

pad_token_id: 128009
eos_token_id: 128009


In [5]:
def generate_response(prompt, json_path):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    print(f"Using device: {device}")

    # Tokenize the input text and create an attention mask
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generate a response using the model
    output = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=300,
        num_return_sequences=1,
        pad_token_id=model.config.pad_token_id  # Use pad_token_id from the model's config
    )

    # Decode and print the output text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Remove the input prompt from the generated response
    generated_text = generated_text[len(prompt)-100:].strip()

    # print("Response:", generated_text)

    # Use regex to extract the JSON block from the response
    json_str_match = re.search(r'{.*}', generated_text, re.DOTALL)

    if json_str_match:
        json_str = json_str_match.group(0)
        # Parse the JSON string
        try:
            json_data = json.loads(json_str)

            # Save the JSON data to a file
            with open(json_path, 'w') as json_file:
                json.dump(json_data, json_file, indent=4)  # Save the JSON with indentation for readability

            # print(f"JSON data saved to '{json_path}'")
            # print("Extracted JSON:", json_data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            print(f"generated text is {generated_text}")
    else:
        print("No JSON found in the response")
        print(f"generated text is {generated_text}")

    del inputs
    del output
    model.cpu()
    torch.cuda.empty_cache()
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    import time


In [6]:
def run_model(content, json_path):
    prompt = f"""
    <|system|> You are a helpful assistant. You process tax records of Norwegian companies and extract specific information from them and insert it in a table with a JSON format.
    This is the tax record you will base your answers on: {content}
    You provide your response only in a predefined JSON format and nothing else. Do not include any explanation, comments, or extra information—only the JSON.
    The JSON structure should strictly follow this format:
    "json"
    {{
        "company_name": "string",
        "company_id": "string",
        "company_address": "string",
        "leadership": {{
            "CEO": "string or null",
            "board_members": [
                "string or null"
            ],
            "chairman_of_the_board": "string or null"
        }},
        "subsidiaries": [
            "string or null"
        ],
        "parent_company": "string or null",
        "property_ownership": {{
        "owns": ["property1", "property2", ...],
        "rents": ["property1", "property2", ...]   }}
        "number_of_employees": "integer or null" ,
        "profit_status": "boolean or null",
    }}
    <|user|> I want to extract the following information from the tax record:
        - "company_name": The name of the company.
        - "company_id": The company tax number.
        - "company_address": The company's address.
        - "leadership": A list of shareholder names. Including a nested dictionary with the CEO, Board members and chairman of the board as follows:
            -"CEO": Name of the CEO.
            -"board_members": A list of the board members.
            -"chairman_of_the_board": Name of the chairman of the board of the company.
        - "subsidiaries": A list of the subsidiaries of the company.
        - "parent_company": Name of the parent company.
        - "property_ownership": A dictionary with two keys, "owns" and "rents", where each contains a list of properties the company owns or rents, respectively. If there are no properties, return an empty list for that category.
        - "number_of_employees": The total number of employees.
        - "profit_status": A bolean indicating whether the company made profits.
    Remember, only return the JSON. No explanations.
    <|assistant|>
    """

    generate_response(prompt, json_path)

In [7]:
def process_md(path):
    match = re.search(r"/(\d+)_arso_2020", path)
    print(match)
    if match:
        id = match.group(1)  # group(1) will contain everything before "_arso_2020"
    else:
        return None
    json_path = f"{json_paths}/{id}.json"
    if not os.path.exists(json_path):
        print(f"processing {path}...")
        with open(path, 'r', encoding='utf-8') as file:
            content = file.read()
        run_model(content, json_path)

In [8]:
import time

for path in markdown_files:
    start_time = time.time()
    # print(f"Processing: {path}")
    process_md(path)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time taken for {path}: {elapsed_time:.2f} seconds")

<re.Match object; span=(33, 53), match='/814377482_arso_2020'>
processing /home/nuno/GAIJ/sandbox/converted/814377482_arso_2020.md...
Using device: cuda


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Time taken for /home/nuno/GAIJ/sandbox/converted/814377482_arso_2020.md: 16.06 seconds
<re.Match object; span=(33, 53), match='/857507622_arso_2020'>
processing /home/nuno/GAIJ/sandbox/converted/857507622_arso_2020.md...
Using device: cuda
Time taken for /home/nuno/GAIJ/sandbox/converted/857507622_arso_2020.md: 99.21 seconds
<re.Match object; span=(33, 53), match='/821560462_arso_2020'>
processing /home/nuno/GAIJ/sandbox/converted/821560462_arso_2020.md...
Using device: cuda
Time taken for /home/nuno/GAIJ/sandbox/converted/821560462_arso_2020.md: 20.48 seconds
<re.Match object; span=(33, 53), match='/910128493_arso_2020'>
processing /home/nuno/GAIJ/sandbox/converted/910128493_arso_2020.md...
Using device: cuda
Time taken for /home/nuno/GAIJ/sandbox/converted/910128493_arso_2020.md: 19.03 seconds
<re.Match object; span=(33, 53), match='/816273072_arso_2020'>
processing /home/nuno/GAIJ/sandbox/converted/816273072_arso_2020.md...
Using device: cuda
Time taken for /home/nuno/GAIJ/sandbox/c

KeyboardInterrupt: 